<a href="https://colab.research.google.com/github/ProjectInvent1A/AIStudioProject/blob/main/Development%20Process/location_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import requests
import json
import torch

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/BTT Project Invent Team 1A/Zoe's API work/df_api_final.csv"
df = pd.read_csv(path)
print(df.shape)

<ipython-input-3-cf49f1fdc9ca>:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


(963062, 10)


In [ ]:
!pip install geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
from geopy.distance import geodesic

#initializes the geolocator
geolocator = Nominatim(user_agent="cities matching")

In [ ]:
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

In [ ]:
# save list of unique cities in a new dataframe
print(df["city"].nunique())
df_cities = pd.DataFrame()
cities_list = df["city"].tolist()
df_cities["city"] = [s.upper() for s in cities_list]
states_list = df["state"].tolist()
df_cities["state"] = [s.upper() for s in states_list]

df_cities.head()

23001


,city,state
0,HEWLETT,NY
1,ELMSFORD,NY
2,FT LAUDERDALE,FL
3,STAMFORD,CT
4,TAMPA,FL


In [ ]:
df_cities["city"].nunique()

20968

In [ ]:
df_cities = df_cities.drop_duplicates(subset='city')
df_cities.shape

(20968, 2)

Get latitude and longitude coordinates for each unique city in our dataset

In [ ]:
coordinates = []
i = 0;
for row in df_cities.itertuples():
  i = i+1;
  if(i % 100 == 0):
    print(str(i))
  try:
    location = geolocator.geocode(f"{row.city}, {row.state}, USA", timeout = 5)
    if location:
      coordinates.append((location.latitude, location.longitude))
    else:
      coordinates.append(None)
  except (GeocoderTimedOut, GeocoderServiceError) as e:
    # Handle timeout or service errors
    print(f"Error for {row.city}, {row.state}: {e}")
    coordinates.append(None)

100
200
300


400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
1870

In [ ]:
print(len(coordinates))
coordinates_copy = coordinates

20968


Save coordinates in the cities dataframe

In [ ]:
df_cities["coordinates"] = coordinates
df_cities.head(50)

,city,state,coordinates
0,HEWLETT,NY,"(40.6431591, -73.6956865)"
1,ELMSFORD,NY,"(41.0550969, -73.8201338)"
2,FT LAUDERDALE,FL,"(26.1223084, -80.1433786)"
3,STAMFORD,CT,"(41.0534302, -73.5387341)"
4,TAMPA,FL,"(27.9477595, -82.458444)"
5,BRONX,NY,"(40.8466508, -73.8785937)"
6,NEW YORK CITY,NY,"(40.7127281, -74.0060152)"
7,DIXFIELD,ME,"(44.5312668, -70.4569591)"
8,LONG ISLAND CITY,NY,"(40.7455316, -73.9484995)"
9,ROCKAWAY PARK,NY,"(40.5805104, -73.8361535)"


In [ ]:
df_cities.tail(50)

,city,state,coordinates
953752,ST LOUIS PARK,MN,"(44.9475726, -93.3569023)"
953773,ST HELENS,OR,"(45.8640034, -122.8064922)"
953871,MT STERLING,KY,"(38.0561079, -83.9431961)"
954106,MT PLEASANT,MI,"(43.597646, -84.7668495)"
954114,CROTONONHUDSON,NY,None
954634,ST JOSEPH,MI,"(41.9031963, -85.5356726)"
954902,ST MARYS,PA,"(41.4280432, -78.5611275)"
954978,MINEAPOLIS,MN,"(44.9772995, -93.2654692)"
955162,INEW HAVEN I,CT,None
955622,MT MORRIS,NY,"(42.7256184, -77.8741695)"


In [ ]:
# looking at cities where geopy couldn't find coordinates -- these look to be mispellings or abbreviations of city names
print(df_cities[df_cities["coordinates"].isna()])
df_cities = df_cities.dropna(subset=['coordinates'])

                 city state coordinates
83      BERRIEN SPRGS    MI        None
111        QUEENS VLG    NY        None
118     FREDERICKSBRG    VA        None
146     CPE GIRARDEAU    MO        None
186     SANTA RSA BCH    FL        None
...               ...   ...         ...
960075    LTL ORLEANS    MD        None
960785  ST STEPHNS CH    VA        None
961087          RACIN    WI        None
961099         FARELL    PA        None
962429  ST CLR SHORES    MI        None

[2136 rows x 3 columns]


In [ ]:
print(df_cities[df_cities["coordinates"].isna()])

Empty DataFrame
Columns: [city, state, coordinates]
Index: []


In [ ]:
df_cities.to_csv('df_coordinates_cleaned.csv', index=False)

In [ ]:
df_cities.head(50)

,city,state,coordinates,nearby_cities,distances
0,HEWLETT,NY,"(40.6431591, -73.6956865)",,
1,ELMSFORD,NY,"(41.0550969, -73.8201338)",,
2,FT LAUDERDALE,FL,"(26.1223084, -80.1433786)",,
3,STAMFORD,CT,"(41.0534302, -73.5387341)",,
4,TAMPA,FL,"(27.9477595, -82.458444)",,
5,BRONX,NY,"(40.8466508, -73.8785937)",,
6,NEW YORK CITY,NY,"(40.7127281, -74.0060152)",,
7,DIXFIELD,ME,"(44.5312668, -70.4569591)",,
8,LONG ISLAND CITY,NY,"(40.7455316, -73.9484995)",,
9,ROCKAWAY PARK,NY,"(40.5805104, -73.8361535)",,


In the following cells experimented with preloading what cities are nearby each city. This ended up not being very efficient, so we didn't end up using it in the final application. Instead, we just used the above dataframe of cities and location coordinates and calculated nearby cities in real time.

In [ ]:
# load in dataframe with coordinates for each city
path = "/content/drive/My Drive/BTT Project Invent Team 1A/Zoe's API work/df_coordinates_cleaned.csv"
df_cities = pd.read_csv(path)
df_cities.shape

(18832, 3)

In [ ]:
coords = df_cities[df_cities["city"] == "HEWLETT"]["coordinates"][0]
coordinates = tuple(map(float, coords.strip("()").split(',')))
print(coordinates[1])

-73.6956865


In [ ]:
def find_nearby_cities(city, coordinates, cities, threshold=15):
  coords = tuple(map(float, coordinates.strip("()").split(',')))
  if not coords:
    print(f"City {city} not found.")
  else:
    print(city)

  nearby_cities = []
  distances = []

  # Calculate distance to each other city
  for other_city, other_coordinates in cities:
    if other_city != city:
      other_coords = tuple(map(float, other_coordinates.strip("()").split(',')))
      distance = geodesic(coords, other_coords).miles
      if distance <= threshold:
        distances.append((other_city, distance))
        nearby_cities.append(other_city)

  return nearby_cities, distances

In [ ]:
df_cities_1000 = df_cities[:1000].copy()

df_cities_1000["nearby_cities"] = ""
df_cities_1000["distances"] = ""
cities_list = list(zip(df_cities['city'], df_cities['coordinates']))

def compute_nearby_cities(row):
    nearby_cities, distances = find_nearby_cities(row["city"], row["coordinates"], cities_list)
    return pd.Series({'nearby_cities': nearby_cities, 'distances': distances})

# Apply the function to each row
df_cities_1000[['nearby_cities', 'distances']] = df_cities_1000.apply(compute_nearby_cities, axis=1)

HEWLETT
ELMSFORD
FT LAUDERDALE
STAMFORD
TAMPA
BRONX
NEW YORK CITY
DIXFIELD
LONG ISLAND CITY
ROCKAWAY PARK
INGLEWOOD
HUDSON
WEST HAVEN
MOUNT KISCO
QUEENS VILLAGE
GROTON
FLUSHING
NEW YORK
JAMAICA
ASTORIA
BALTIMORE
MOUNT VERNON
SPRINGFIELD GDNS
STATEN ISLAND
BROOKLYN
BOSTON
MILFORD
SOUTHPORT
NASHVILLE
NEW HAVEN
PROVIDENCE
PORTLAND
ASHEVILLE
SEYMOUR
BALDWIN
BRUNSWICK
W BARNSTABLE
NEW SALEM
WINNSBORO
PITTSFIELD
WILLSBORO
CORTLAND
MIDDLETOWN
WINCHESTER
BARKHAMSTED
EAST MEADOW
PLANTERSVILLE
LAS CRUCES
HOPE MILLS
HOPKINTON
PITTSBURGH
ATHOL
ARDMORE
ALBUQUERQUE
WOBURN
BRANDON
MEDWAY
BRIGHTON
BROWNSVILLE
SPRINGTOWN
WOODBURY
ORLANDO
GARLAND
COMMACK
ROCKMART
SPRINGFIELD
FAIRVIEW PARK
NEW BEDFORD
INMAN
BRENTWOOD
DURANGO
FRESNO
VALLEY STREAM
BEVERLY
LEBANON
WESTPORT
SYOSSET
CAMBRIDGE
BUXTON
WINDHAM
WEST PALM BEACH
OXON HILL
CINCINNATI
YELM
RED ROCK
SAN FRANCISCO
MADISON
DORCHESTER
MARLBOROUGH
BLAUVELT
CONCORD
BASKING RIDGE
CONGERS
UPPER DARBY
LAUDERHILL
NYC
NEW LONDON
DENVILLE
JACKSON HEIGHTS
E SPRIN

In [ ]:
df_cities_1000.head(50)

,city,state,coordinates,nearby_cities,distances
0,HEWLETT,NY,"(40.6431591, -73.6956865)","[ELMSFORD, STAMFORD, BRONX, NEW YORK CITY, LON...","[(ELMSFORD, 29.163653352427367), (STAMFORD, 29..."
1,ELMSFORD,NY,"(41.0550969, -73.8201338)","[HEWLETT, STAMFORD, BRONX, NEW YORK CITY, LONG...","[(HEWLETT, 29.163653352427367), (STAMFORD, 14...."
2,FT LAUDERDALE,FL,"(26.1223084, -80.1433786)","[LAUDERHILL, MIRAMAR, BOCA RATON, NORTH MIAMI,...","[(LAUDERHILL, 4.524093578102845), (MIRAMAR, 10..."
3,STAMFORD,CT,"(41.0534302, -73.5387341)","[HEWLETT, ELMSFORD, BRONX, MOUNT KISCO, QUEENS...","[(HEWLETT, 29.480641707420695), (ELMSFORD, 14...."
4,TAMPA,FL,"(27.9477595, -82.458444)","[ZEPHYRHILLS, LUTZ, RIVERVIEW, DUNEDIN, PORT R...","[(ZEPHYRHILLS, 25.964255266653602), (LUTZ, 12...."
5,BRONX,NY,"(40.8466508, -73.8785937)","[HEWLETT, ELMSFORD, STAMFORD, NEW YORK CITY, L...","[(HEWLETT, 17.008920040221202), (ELMSFORD, 14...."
6,NEW YORK CITY,NY,"(40.7127281, -74.0060152)","[HEWLETT, ELMSFORD, BRONX, LONG ISLAND CITY, R...","[(HEWLETT, 16.994497665454016), (ELMSFORD, 25...."
7,DIXFIELD,ME,"(44.5312668, -70.4569591)","[ALBANY TWP, NORWAY, WAYNE, GREENE, WINTHROP, ...","[(ALBANY TWP, 22.125390713901897), (NORWAY, 22..."
8,LONG ISLAND CITY,NY,"(40.7455316, -73.9484995)","[HEWLETT, ELMSFORD, BRONX, NEW YORK CITY, ROCK...","[(HEWLETT, 15.039831015397413), (ELMSFORD, 22...."
9,ROCKAWAY PARK,NY,"(40.5805104, -73.8361535)","[HEWLETT, BRONX, NEW YORK CITY, LONG ISLAND CI...","[(HEWLETT, 8.558380340888823), (BRONX, 18.4989..."


In [ ]:
df_cities.to_csv('df_cities_1000_near.csv', index=False)

In [ ]:
df_cities_temp.head(30)

,city,state,coordinates,nearby_cities,distances
0,HEWLETT,NY,"(40.6431591, -73.6956865)","[ELMSFORD, STAMFORD, BRONX, NEW YORK CITY, LON...","[(ELMSFORD, 29.163653352427367), (STAMFORD, 29..."
1,ELMSFORD,NY,"(41.0550969, -73.8201338)","[HEWLETT, STAMFORD, BRONX, NEW YORK CITY, LONG...","[(HEWLETT, 29.163653352427367), (STAMFORD, 14...."
2,FT LAUDERDALE,FL,"(26.1223084, -80.1433786)",[],[]
3,STAMFORD,CT,"(41.0534302, -73.5387341)","[HEWLETT, ELMSFORD, BRONX, MOUNT KISCO, QUEENS...","[(HEWLETT, 29.480641707420695), (ELMSFORD, 14...."
4,TAMPA,FL,"(27.9477595, -82.458444)",[],[]
5,BRONX,NY,"(40.8466508, -73.8785937)","[HEWLETT, ELMSFORD, STAMFORD, NEW YORK CITY, L...","[(HEWLETT, 17.008920040221202), (ELMSFORD, 14...."
6,NEW YORK CITY,NY,"(40.7127281, -74.0060152)","[HEWLETT, ELMSFORD, BRONX, LONG ISLAND CITY, R...","[(HEWLETT, 16.994497665454016), (ELMSFORD, 25...."
7,DIXFIELD,ME,"(44.5312668, -70.4569591)",[],[]
8,LONG ISLAND CITY,NY,"(40.7455316, -73.9484995)","[HEWLETT, ELMSFORD, BRONX, NEW YORK CITY, ROCK...","[(HEWLETT, 15.039831015397413), (ELMSFORD, 22...."
9,ROCKAWAY PARK,NY,"(40.5805104, -73.8361535)","[HEWLETT, BRONX, NEW YORK CITY, LONG ISLAND CI...","[(HEWLETT, 8.558380340888823), (BRONX, 18.4989..."
